In [1]:
#!unzip deploy-on-pynq-cnv-72.zip -d finn-cnv-demo-72

In [2]:
#!sudo python3 -m pip install bitstring

In [3]:
!pwd

/home/xilinx/jupyter_notebooks


In [4]:
cd finn_demo_GTSRB

/home/xilinx/jupyter_notebooks/finn_demo_GTSRB


In [5]:
!ls

driver_base.py	input.npy	__pycache__  resizer.hwh	 validate.py
driver.py	nw_metrics.txt	qonnx	     runtime_weights
finn		output.npy	resizer.bit  sds_trace_data.dat


In [6]:
#!sudo python3 driver.py --exec_mode=execute --batchsize=1 --bitfile=resizer.bit --inputfile=input.npy

In [7]:
#!sudo pip3 install git+https://github.com/fbcotter/dataset_loading.git@0.0.4#egg=dataset_loading

In [16]:
#!sudo python3 validate.py --dataset cifar10 --batchsize 1000 --bitfile=resizer.bit

Tar File found in dest_dir. Not Downloading again
Extracting Python CIFAR10 data.
Files extracted
shape: (10000, 32, 32, 3)
total: 10000, bsize: 1000 n_batches: 10
batch 1 / 10 : total OK 20 NOK 980
batch 2 / 10 : total OK 39 NOK 1961
batch 3 / 10 : total OK 52 NOK 2948
batch 4 / 10 : total OK 67 NOK 3933
batch 5 / 10 : total OK 83 NOK 4917
batch 6 / 10 : total OK 99 NOK 5901
batch 7 / 10 : total OK 122 NOK 6878
batch 8 / 10 : total OK 141 NOK 7859
batch 9 / 10 : total OK 167 NOK 8833
batch 10 / 10 : total OK 182 NOK 9818
Final accuracy: 1.820000


In [24]:
import numpy as np
from driver import io_shape_dict
from driver_base import FINNExampleOverlay

bitfile = 'resizer.bit'
platform = 'zynq-iodma'
bsize = 421

driver = FINNExampleOverlay(
    bitfile_name=bitfile,
    platform=platform,
    io_shape_dict=io_shape_dict,
    batch_size=bsize,
    runtime_weight_dir="runtime_weights/",
)

In [25]:
from os import path
import urllib
import urllib.request
import numpy as np
dataset_local = "/tmp/traffic-signs-data.zip"
if not path.isfile(dataset_local):
    dataset_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip"
    urllib.request.urlretrieve(dataset_url, dataset_local)
    ! unzip {dataset_local} -d /tmp

dataset_dict = np.load("/tmp/test.p", allow_pickle=True)
testx = dataset_dict["features"]
testy = dataset_dict["labels"]

In [26]:
test_imgs = testx
test_labels = testy

ok = 0
nok = 0
total = test_imgs.shape[0]

n_batches = int(total / bsize)
print(f"total: {total}, bsize: {bsize} n_batches: {n_batches}")
test_imgs = test_imgs.reshape(n_batches, bsize, -1)
test_labels = test_labels.reshape(n_batches, bsize)

for i in range(n_batches):
    ibuf_normal = test_imgs[i].reshape(driver.ibuf_packed_device[0].shape)
    exp = test_labels[i]
    driver.copy_input_data_to_device(ibuf_normal)
    driver.execute_on_buffers()
    obuf_normal = np.empty_like(driver.obuf_packed_device[0])
    driver.copy_output_data_from_device(obuf_normal)
    ret = np.bincount(obuf_normal.flatten() == exp.flatten())
    nok += ret[0]
    ok += ret[1]
    print("batch %d / %d : total OK %d NOK %d" % (i + 1, n_batches, ok, nok))

acc = 100.0 * ok / (total)
print("Final accuracy: %f" % acc)

total: 12630, bsize: 421 n_batches: 30
batch 1 / 30 : total OK 399 NOK 22
batch 2 / 30 : total OK 805 NOK 37
batch 3 / 30 : total OK 1209 NOK 54
batch 4 / 30 : total OK 1615 NOK 69
batch 5 / 30 : total OK 2016 NOK 89
batch 6 / 30 : total OK 2417 NOK 109
batch 7 / 30 : total OK 2822 NOK 125
batch 8 / 30 : total OK 3229 NOK 139
batch 9 / 30 : total OK 3631 NOK 158
batch 10 / 30 : total OK 4039 NOK 171
batch 11 / 30 : total OK 4435 NOK 196
batch 12 / 30 : total OK 4836 NOK 216
batch 13 / 30 : total OK 5235 NOK 238
batch 14 / 30 : total OK 5630 NOK 264
batch 15 / 30 : total OK 6032 NOK 283
batch 16 / 30 : total OK 6433 NOK 303
batch 17 / 30 : total OK 6834 NOK 323
batch 18 / 30 : total OK 7239 NOK 339
batch 19 / 30 : total OK 7644 NOK 355
batch 20 / 30 : total OK 8041 NOK 379
batch 21 / 30 : total OK 8444 NOK 397
batch 22 / 30 : total OK 8853 NOK 409
batch 23 / 30 : total OK 9265 NOK 418
batch 24 / 30 : total OK 9673 NOK 431
batch 25 / 30 : total OK 10074 NOK 451
batch 26 / 30 : total OK 1

In [28]:
driver.batch_size = 1000
driver.throughput_test()

{'DRAM_in_bandwidth[MB/s]': 5.698918717806503,
 'DRAM_out_bandwidth[MB/s]': 0.0018551167701193045,
 'batch_size': 1000,
 'copy_input_data_to_device[ms]': 22.85146713256836,
 'copy_output_data_from_device[ms]': 0.21004676818847656,
 'fclk[mhz]': 100.0,
 'fold_input[ms]': 0.13685226440429688,
 'pack_input[ms]': 0.10061264038085938,
 'runtime[ms]': 539.0496253967285,
 'throughput[images/s]': 1855.1167701193044,
 'unfold_output[ms]': 0.1964569091796875,
 'unpack_output[ms]': 0.9667873382568359}